<a href="https://colab.research.google.com/github/IgorekFromRU/RecSys/blob/master/RecSys_unit6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Построение рекомендательных систем.

In [1]:
from google.colab import drive
# Монтируем Google Drive
drive.mount('/content/drive')
# Устанавливаем рабочий каталог
import os
os.chdir('/content/drive/MyDrive/DS/')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import math

In [4]:
articles_df = pd.read_csv('data/shared_articles.zip')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [5]:
interactions_df = pd.read_csv('data/users_interactions.zip')
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [6]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [7]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0,
   'BOOKMARK': 2.5,
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,
}

In [8]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

In [9]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

In [10]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [11]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)

interactions_full_df = interactions_full_df.reset_index()

In [12]:
from sklearn.model_selection import train_test_split

split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


Для начала необходимо построить матрицу, в которой по столбцам будут находиться id статей, по строкам — id пользователей, а на пересечениях строк и столбцов — оценка взаимодействия пользователя со статьёй. Если взаимодействия не было, в соответствующей ячейке должен стоять ноль.

## Задание 6.1

Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. Результат округлите до двух знаков после точки-разделителя.

Примечание. Здесь и далее (пока не будет указано иное) необходимо работать с обучающей выборкой.

In [13]:
ratings = pd.pivot_table(
    interactions_train_df,
    values="eventStrength",
    index="personId",
    columns="contentId",
).fillna(0)
round(ratings.loc["-1032019229384696495", "943818026930898372"], 2)

2.32

Теперь давайте попробуем применить memory-based-подход коллаборативной фильтрации.

Примечание. Данных достаточно много, поэтому для увеличения скорости работы преобразуйте таблицу в массив numpy.

## Задание 6.2

Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

In [14]:
ratings_m = ratings.values
ratings_m.mean().round(3)

0.017

Перейдём к реализации коллаборативной фильтрации. Ранее мы делали это с помощью библиотеки surprise, однако это не всегда удобно, так как эта библиотека имеет ограниченное количество метрик для оценки качества и небольшой потенциал для более тонкой настройки алгоритма. Поэтому давайте попробуем реализовать алгоритмы коллаборативной фильтрации «с нуля». Такая практика применяется, если необходимо выстроить более сложную систему, чем могут предложить готовые модули. Кроме того, «ручная» реализация алгоритмов позволит лучше понять принцип их работы.

## Задание 6.3

Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам. Ответ округлите до двух знаков после точки-разделителя.

In [16]:
# Создание матрицы схожести
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

# Вычисление схожести между парами пользователей
for i in range(len(ratings_m) - 1):
    for j in range(i + 1, len(ratings_m)):
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        # Проверка наличия ненулевых значений
        if len(ratings_v) > 0 and len(ratings_u) > 0:
            similarity_users[i, j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
            similarity_users[j, i] = similarity_users[i, j]

# Вывод результата для ячейки (3, 40)
print("Результат для ячейки (3, 40):", round(similarity_users[3, 40], 2))


Результат для ячейки (3, 40): -0.33


Теперь у нас есть матрицы схожести пользователей. Их можно использовать для построения рекомендаций. Чтобы это сделать, надо реализовать следующий алгоритм.

Для каждого пользователя:

Найти пользователей с похожестью больше 0.
Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

## Задание 6.4

Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [17]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]

prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))

interactions['prediction_user_based'] = prediction_user_based
prediction_user_based[35][0]

'-5148591903395022444'

После того как сделаны предсказания, можно вычислить качество по метрике, которую мы определили в предыдущем модуле при решении этой задачи:

In [18]:
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

## Задание 6.5

Вычислите точность полученного предсказания. Ответ округлите до трёх знаков после точки-разделителя.

In [19]:
calc_precision('prediction_user_based').round(3)

0.005

## Задание 6.6

Теперь реализуем рекомендательную систему с использованием SVD.

Разложите матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scipy. Найдите максимальное значение в получившейся матрице U. Результат округлите до двух знаков после точки-разделителя.

scipy.linalg.svd(a, full_matrices=True, compute_uv=True, overwrite_a=False, check_finite=True, lapack_driver='gesdd')[source]
Singular Value Decomposition.

Factorizes the matrix a into two unitary matrices U and Vh, and a 1-D array s of singular values (real, non-negative) such that a == U @ S @ Vh, where S is a suitably shaped matrix of zeros with main diagonal s.

In [21]:
from scipy.linalg import svd

U, sigma, V = svd(ratings)
U.max().round(2)

0.71

Значения матрицы с сингулярными числами отсортированы по убыванию. Допустим, мы хотим оставить только первые 100 компонент и получить скрытые представления размерности 100. Для этого необходимо оставить 100 столбцов в матрице U, только первые 100 значений из sigma (и сделать из них диагональную матрицу) и 100 строк в матрице V. Затем необходимо перемножить преобразованные матрицы.

## Задание 6.7

Найдите сумму всех элементов в новой сингулярной матрице. Ответ округлите до двух знаков после точки-разделителя.

In [22]:
k = 100
s = np.diag(sigma[:k])
U = U[:, 0:k]
V = V[0:k, :]

result = round(s.sum(), 2)
print("Сумма всех элементов в новой сингулярной матрице:", result)


Сумма всех элементов в новой сингулярной матрице: 2096.43


Теперь мы можем сделать предсказание по полученной матрице.

Примечание. Помните, что не нужно учитывать статьи, которые уже были просмотрены пользователем.

Найдите для каждого пользователя статьи с наибольшими оценками в восстановленной матрице.

## Задание 6.8

Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-разделителя.

In [23]:
new_ratings = pd.DataFrame(
    U.dot(s).dot(V), index=ratings.index, columns=ratings.columns
)
top_k = 10
predictions = []

for personId in interactions.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                ~np.in1d(prediction, interactions.loc[personId, "true_train"])
            ]
        )[:top_k]
    )

interactions["prediction_svd"] = predictions

calc_precision("prediction_svd").round(3)

0.012

Итак, мы реализовали два алгоритма коллаборативной фильтрации буквально с нуля! Теперь для полноты картины давайте реализуем на этих данных гибридную модель и посмотрим, какое качество получится. Для этого воспользуемся уже изученной библиотекой **LightFM**.

## Задание 6.9

Возьмите матрицу, подготовленную в задании 6.1. Преобразуйте её в разреженную матрицу:

In [24]:
from scipy.sparse import csr_matrix
ratings_matrix = csr_matrix(ratings)

Воспользовавшись функцией *random_train_test_split()* из библиотеки **lightfm**, разделите данные на валидационную и обучающую выборки в соотношении 1:2 (30% на валидационную выборку, 70% на обучающую). В качестве значения параметра *random_state* возьмите число 13.

Обучите модель **LightFM** со 100 компонентами, параметром *random_state* = 13, темпом обучения 0.05 и функцией потерь *'warp'*. Обратите внимание на то, что так как в данном случае у нас нет item-признаков, то параметр item_features задавать не нужно.

Вычислите показатель точности (*precision@k*) при k = 10. Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
!pip install lightfm

In [29]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from scipy.sparse import csr_matrix

ratings_matrix = csr_matrix(ratings) # передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)
model = LightFM(loss='warp', #определяем функцию потерь
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели

train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)
model.fit(train)

prec_score = precision_at_k(model, test, k=10).mean()
print(f"Показатель точности (precision@10): {prec_score:.2f}")

Показатель точности (precision@10): 0.04


В данном случае модель «из коробки» показала наилучший результат, однако это совсем не показатель того, что стоит пользоваться исключительно готовыми функциями. Зная тонкости работы алгоритмов, вы можете создавать собственные гибридные системы, настраивать отдельные алгоритмы и добиваться ещё лучших результатов.